## Overview:  
This capstone project is your opportunity to demonstrate the knowledge and abilities you have acquired throughout the course.
This Capstone Project requires learners to work with the following technologies to manage an ETL process for a Loan Application dataset and a Credit Card dataset: Python (Pandas, advanced modules e.g., Matplotlib), MariaDB, Apache Spark (Spark Core, Spark SQL), and Python Visualization and Analytics libraries. Learners are expected to set up their environments and perform installations on their local machines. 


![My Image](workflow.jpg)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType,TimestampType
from datetime import datetime
import pyspark.sql.functions as F
from pyspark.sql.functions import  concat_ws, regexp_replace, concat,lit,col,udf, lpad,when, length,expr,substring
import re
import requests
from pyspark import SparkContext, SparkConf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector
import configparser
import pyinputplus as pyip


In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['DATABASE']['host']
user = config['DATABASE']['user']
password = config['DATABASE']['password']
database = config['DATABASE']['database']

In [4]:
spark = SparkSession.builder.appName("CreditCardSystem").getOrCreate()


In [5]:
branch_df=spark.read.json("cdw_sapp_branch.json")
creditcard_df=spark.read.json("cdw_sapp_credit.json")
customer_df=spark.read.json("cdw_sapp_custmer.json")


### Transform Customer Data frame

In [6]:
# add area code 555 as the first 3 characters for customer phone number, so we can  convert cust_phone column into this format (XXX)XXX-XXXX later
customer_df = customer_df.withColumn("CUST_PHONE", concat(lit("555"), customer_df["CUST_PHONE"]))

In [7]:
customer_df = customer_df.withColumn("cust_phone", 
                   expr("concat('(', substring(cust_phone, 1, 3), ')',  " 
                        "substring(cust_phone, 4, 3), '-', "
                        "substring(cust_phone, 7, 4))").cast("string"))
 
customer_df = customer_df.withColumn("FIRST_NAME", F.initcap("FIRST_NAME").cast("string"))
customer_df = customer_df.withColumn("SSN", customer_df["SSN"].cast("int"))
customer_df = customer_df.withColumn("MIDDLE_NAME", F.lower(customer_df["MIDDLE_NAME"]).cast("string"))
customer_df = customer_df.withColumn("LAST_NAME", F.initcap(customer_df["LAST_NAME"]).cast("string"))
customer_df = customer_df.withColumn("FULL_STREET_ADDRESS", F.concat_ws(", ", customer_df["STREET_NAME"], customer_df["APT_NO"]).cast("string"))
customer_df = customer_df.withColumn("CREDIT_CARD_NO", customer_df["CREDIT_CARD_NO"].cast("string"))
customer_df = customer_df.withColumn("CUST_CITY", customer_df["CUST_CITY"].cast("string"))
customer_df = customer_df.withColumn("CUST_STATE", customer_df["CUST_STATE"].cast("string"))
customer_df = customer_df.withColumn("CUST_COUNTRY", customer_df["CUST_COUNTRY"].cast("string"))
customer_df = customer_df.withColumn("CUST_ZIP", customer_df["CUST_ZIP"].cast("int"))

 
#customer_df = customer_df.withColumn("CUST_PHONE", F.regexp_replace(customer_df["CUST_PHONE"], r'^(\d{3})(\d{3})(\d{4})$', r'(\1)\2-\3').cast("string"))
customer_df = customer_df.withColumn("CUST_EMAIL", customer_df["CUST_EMAIL"].cast("string"))
customer_df = customer_df.withColumn("LAST_UPDATED", customer_df["LAST_UPDATED"].cast("timestamp"))

customer_df=customer_df.select("SSN", "FIRST_NAME","MIDDLE_NAME","LAST_NAME","CREDIT_CARD_NO","FULL_STREET_ADDRESS","CUST_CITY", "CUST_STATE","CUST_COUNTRY","CUST_ZIP","CUST_PHONE","CUST_EMAIL","LAST_UPDATED")

In [8]:
customer_df.show(5)


+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO| FULL_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|   CUST_PHONE|         CUST_EMAIL|       LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+-------------+-------------------+-------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|Main Street North...|     Natchez|        MS|United States|   39120|(555)123-7818|AHooper@example.com|2018-04-21 12:49:02|
|123453023|      Etta|    brendan|   Holman|4210653310102868|  Redwood Drive, 829|Wethersfield|        CT|United States|    6109|(555)123-8933|EHolman@example.com|2018-04-21 12:49:02|
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Street East,..

In [ ]:
customer_df.printSchema()

### transform branch data frame

In [9]:
# Convert column data types


branch_df = branch_df.withColumn("BRANCH_PHONE", 
                   expr("concat('(', substring(BRANCH_PHONE, 1, 3), ')',  " 
                        "substring(BRANCH_PHONE, 4, 3), '-', "
                        "substring(BRANCH_PHONE, 7, 4))").cast("string"))

branch_df = branch_df.withColumn("BRANCH_CODE", branch_df["BRANCH_CODE"].cast("int"))
branch_df = branch_df.withColumn("BRANCH_NAME", branch_df["BRANCH_NAME"].cast("string"))
branch_df = branch_df.withColumn("BRANCH_STREET", branch_df["BRANCH_STREET"].cast("string"))
branch_df = branch_df.withColumn("BRANCH_CITY", branch_df["BRANCH_CITY"].cast("string"))
branch_df = branch_df.withColumn("BRANCH_STATE", branch_df["BRANCH_STATE"].cast("string"))
branch_df = branch_df.withColumn("BRANCH_ZIP", F.when(branch_df["BRANCH_ZIP"].isNull(), 99999).otherwise(branch_df["BRANCH_ZIP"].cast("int")))
#branch_df = branch_df.withColumn("BRANCH_PHONE", F.regexp_replace(branch_df["BRANCH_PHONE"], r"^(\d{3})(\d{3})(\d{4})$", r"(\1)\2-\3"))
branch_df = branch_df.withColumn("LAST_UPDATED", branch_df["LAST_UPDATED"].cast("timestamp"))

# Select columns 
branch_df = branch_df.select("BRANCH_CODE", "BRANCH_NAME", "BRANCH_STREET", "BRANCH_CITY", "BRANCH_STATE", "BRANCH_ZIP", "BRANCH_PHONE", "LAST_UPDATED")

In [ ]:
branch_df.printSchema()

In [10]:
branch_df.show(5)

+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|       LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+-------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18 16:51:47|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18 16:51:47|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18 16:51:47|
|          4|Example Bank| Cleveland Street|       Middleburg|          FL|     32068|(123)466-3064|2018-04-18 16:51:47|
|          5|Example Bank|      14th Street|    KingOfPrussia|          PA|     19406|(123)484-9701|2018-04-18 16:51:47|
+-----------+------------+------

### TRANSFORM CREDIT CARD DATA 

In [11]:
# Convert DAY, MONTH and YEAR into date format and create new column TIMEID with YYYYMMDD format
creditcard_df = creditcard_df.withColumn('TIMEID', concat(creditcard_df.YEAR, lpad(creditcard_df.MONTH, 2, '0'), lpad(creditcard_df.DAY, 2, '0')).cast('int'))


In [12]:
 


# Rename CREDIT_CARD_NO column as per the mapping document
creditcard_df = creditcard_df.withColumnRenamed('CREDIT_CARD_NO', 'CUST_CC_NO') 

# Convert datatype of columns as per the mapping document
creditcard_df = creditcard_df.withColumn('CUST_CC_NO', creditcard_df['CUST_CC_NO'].cast('string')) 
creditcard_df = creditcard_df.withColumn('CUST_SSN', creditcard_df['CUST_SSN'].cast('int')) 
creditcard_df = creditcard_df.withColumn('BRANCH_CODE', creditcard_df['BRANCH_CODE'].cast('int')) 
creditcard_df = creditcard_df.withColumn('TRANSACTION_TYPE', creditcard_df['TRANSACTION_TYPE'].cast('string')) 
creditcard_df = creditcard_df.withColumn('TRANSACTION_VALUE', creditcard_df['TRANSACTION_VALUE'].cast('double')) 
creditcard_df = creditcard_df.withColumn('TRANSACTION_ID', creditcard_df['TRANSACTION_ID'].cast('int')) 
creditcard_df = creditcard_df.withColumn('TIMEID', creditcard_df['TIMEID'].cast('string'))
creditcard_df=creditcard_df.select("CUST_CC_NO","TIMEID","CUST_SSN","BRANCH_CODE","TRANSACTION_TYPE","TRANSACTION_VALUE","TRANSACTION_ID")
 


In [ ]:
creditcard_df.printSchema()

In [13]:
creditcard_df.show(10)

+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653349028689|20180214|123459988|        114|       Education|             78.9|             1|
|4210653349028689|20180320|123459988|         35|   Entertainment|            14.24|             2|
|4210653349028689|20180708|123459988|        160|         Grocery|             56.7|             3|
|4210653349028689|20180419|123459988|        114|   Entertainment|            59.73|             4|
|4210653349028689|20181010|123459988|         93|             Gas|             3.59|             5|
|4210653349028689|20180528|123459988|        164|       Education|             6.89|             6|
|4210653349028689|20180519|123459988|        119|   Entertainment|            43.39|             7|


### Data loading into Database

In [14]:
# Connect to MariaDB server
cnx = mysql.connector.connect(user=user, password=password,
                              host=host)

# Create database
cursor = cnx.cursor()
DB_NAME = 'creditcard_capstone'
cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))

In [15]:
# Connect to mariadb

try:
    cnx = mysql.connector.connect(user=user, password=password,
                              host=host,
                              database=database)
    cursor = cnx.cursor()

except mysql.connector.Error as err:
    print("Error connecting to MariaDB: {}".format(err))
    exit()

In [16]:
try:
    cursor.execute("""CREATE TABLE IF NOT EXISTS CDW_SAPP_BRANCH (
        BRANCH_CODE INT PRIMARY KEY, 
        BRANCH_NAME VARCHAR(50) NOT NULL, 
        BRANCH_STREET VARCHAR(100) NOT NULL, 
        BRANCH_CITY VARCHAR(50) NOT NULL, 
        BRANCH_STATE VARCHAR(50) NOT NULL, 
        BRANCH_ZIP INT NOT NULL, 
        BRANCH_PHONE VARCHAR(20) NOT NULL, 
        LAST_UPDATED TIMESTAMP NOT NULL)""")
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS CDW_SAPP_CREDIT_CARD (
        CUST_CC_NO VARCHAR(50) PRIMARY KEY, 
        TIMEID VARCHAR(8)NOT NULL, 
        CUST_SSN INT NOT NULL, 
        BRANCH_CODE INT NOT NULL, 
        TRANSACTION_TYPE VARCHAR(20) NOT NULL, 
        TRANSACTION_VALUE DOUBLE NOT NULL, 
        TRANSACTION_ID INT NOT NULL)""")
    
    cursor.execute("""CREATE TABLE IF NOT EXISTS CDW_SAPP_CUSTOMER (
        SSN INT PRIMARY KEY, 
        CREDIT_CARD_NO VARCHAR(50) NOT NULL, 
        CUST_FIRST_NAME VARCHAR(50) NOT NULL, 
        CUST_MIDDLE_NAME VARCHAR(50) NOT NULL, 
        CUST_LAST_NAME VARCHAR(50) NOT NULL, 
        CUST_STREET_ADDRESS VARCHAR(100) NOT NULL, 
        CUST_APT_NO VARCHAR(10) NOT NULL, 
        CUST_CITY VARCHAR(50) NOT NULL, 
        CUST_STATE VARCHAR(50) NOT NULL, 
        CUST_COUNTRY VARCHAR(50) NOT NULL, 
        CUST_ZIP INT NOT NULL, 
        CUST_PHONE VARCHAR(10) NOT NULL, 
        CUST_EMAIL VARCHAR(50) NOT NULL, 
        LAST_UPDATED TIMESTAMP NOT NULL)""")
    
except mysql.connector.Error as err:
    print("Error creating tables: {}".format(err))
    exit()


In [20]:
# Load the data into the database tables
try:
    branch_df.write.jdbc(url='jdbc:mysql://localhost:3306/creditcard_capstone', table='CDW_SAPP_BRANCH', mode='overwrite', properties={'user': user, 'password': password})
    creditcard_df.write.jdbc(url='jdbc:mysql://localhost:3306/creditcard_capstone', table='CDW_SAPP_CREDIT_CARD', mode='overwrite', properties={'user': user, 'password': password})
    customer_df.write.jdbc(url='jdbc:mysql://localhost:3306/creditcard_capstone', table='CDW_SAPP_CUSTOMER', mode='overwrite', properties={'user': user, 'password': password})
except mysql.connector.Error as err:
    print("Error loading data: {}".format(err))
    exit()


In [21]:
# Close the database and cursor connection
cursor.close()
cnx.close()

### 2. Functional Requirements - Application Front-End

 ##### Customer Details menu

In [ ]:


def check_customer_details():
    # code to check existing account details of a customer
    pass

def modify_customer_details():
    # code to modify existing account details of a customer
    pass

def generate_monthly_bill():
    # code to generate monthly bill for a credit card number for a given month and year
    pass

def display_customer_transactions():
    # code to display transactions made by a customer between two dates, ordered by year, month, and day in descending order
    pass

while True:
    print("Please select an option:")
    print("1. Check customer details")
    print("2. Modify customer details")
    print("3. Generate monthly bill")
    print("4. Display customer transactions")
    print("5. Exit")
    
    choice = pyip.inputInt("Enter your choice: ", min=1, max=5)
    
    if choice == 1:
        check_customer_details()
    elif choice == 2:
        modify_customer_details()
    elif choice == 3:
        generate_monthly_bill()
    elif choice == 4:
        display_customer_transactions()
    elif choice == 5:
        print("Exiting program.")
        break


## Overview of LOAN application Data API
Banks deal in all home loans. They have a presence across all urban, semi-urban, and rural areas. Customers first apply for a home loan; after that, a company will validate the customer's eligibility for a loan.
Banks want to automate the loan eligibility process (in real-time) based on customer details provided while filling out the online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History, and others. To automate this process, they have the task of identifying the customer segments to those who are eligible for loan amounts so that they can specifically target these customers. Here they have provided a partial dataset.


API Endpoint: https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json

The above URL allows you to access information about loan application information. This dataset has all of the required fields for a loan application. You can access data from a REST API by sending an HTTP request and processing the response.


In [ ]:
# Load data from API endpoint into Pandas dataframe
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
response =requests.get(url)
response_json = response.json()


In [ ]:
print("Status code:", response.status_code)

In [ ]:
# Write the DataFrame to a SQL table in the "creditcard_capstone" database


In [ ]:
# Plot the percentage of applications approved for self-employed applicants




In [ ]:
# Plot the percentage of rejection for married male applicants


In [ ]:
# Stop SparkSession
spark.stop()